In [1]:
%pip install -q langchain langchain-community langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -q python-dotenv
from dotenv import load_dotenv
load_dotenv()

Note: you may need to restart the kernel to use updated packages.


True

In [3]:
import yaml

# https://raw.githubusercontent.com/open-meteo/open-meteo/main/openapi.yml
with open("data/open-meteo.yml") as f:
    raw_meteo_api_spec = yaml.load(f, Loader=yaml.Loader)

# https://github.com/langchain-ai/langchain/issues/2456
# in reduce_openapi_spec
#    servers=spec["servers"],
#            ~~~~^^^^^^^^^^^
raw_meteo_api_spec["servers"] = [{"url": "https://api.open-meteo.com"}]
# raw_meteo_api_spec.servers[0]["url"] = "https://api.foo.com"
# print(raw_meteo_api_spec)

from langchain_community.agent_toolkits.openapi.spec import reduce_openapi_spec
meteo_api_spec = reduce_openapi_spec(raw_meteo_api_spec)


In [4]:
from _lessonshelper.pretty_print_callback_handler import PrettyPrintCallbackHandler
pretty_callback = PrettyPrintCallbackHandler()

from langchain_openai import OpenAI
#agent_llm = OpenAI(temperature=0, callbacks=[pretty_callback])
agent_llm = OpenAI(temperature=0)

In [5]:
from langchain.requests import RequestsWrapper
from langchain_community.agent_toolkits.openapi import planner

wrapper = RequestsWrapper()
meteo_agent = planner.create_openapi_agent(
    meteo_api_spec, requests_wrapper=wrapper, llm=agent_llm
)

user_query = "What are the current weather conditions in Ghent expressed in Celsius?"
answer = meteo_agent.run(user_query)
print(answer)

/Users/patrickdebois/Library/Caches/pypoetry/virtualenvs/learning-llms-and-genai-for-dev-sec-ops--fI-1qgv-py3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Action: api_planner
Action Input: I need to find the right API calls to get the current weather conditions in Ghent in Celsius
Observation:  1. GET /v1/forecast with query params for Ghent's latitude and longitude
2. Convert temperature from Kelvin to Celsius
Thought: I'm ready to execute the API calls.
Action: api_controller
Action Input: 1. GET /v1/forecast with query params for Ghent's latitude and longitude
2. Convert temperature from Kelvin to Celsius

> Entering new AgentExecutor chain...
